In [18]:
import numpy as np
from mlxtend.data import loadlocal_mnist
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

In [2]:
xTrain,yTrain=loadlocal_mnist(images_path='./fmnist/train-images-idx3-ubyte', labels_path='./fmnist/train-labels-idx1-ubyte')
xTest,yTest = loadlocal_mnist(images_path='./fmnist/t10k-images-idx3-ubyte', labels_path='./fmnist/t10k-labels-idx1-ubyte')

In [3]:
print(xTrain.shape,yTrain.shape)

(60000, 784) (60000,)


In [15]:
#class for FDA
class FDA:
    W = None
    eigenValues = None
    Sw, Sb = None, None

    def __init__(self):
        W = None
        eigenValues = None
        Sw, Sb = None, None
        
    def fit(self, x, y):
        Sw,Sb = self.getScatters(x, y)
        self.Sw = Sw
        self.Sb = Sb
        Sw_inv_Sb = np.matmul(np.linalg.inv(Sw), Sb)
        eigValues, eigVectors = np.linalg.eigh(Sw_inv_Sb)
        idx = eigValues.argsort()[::-1]   
        eigVectors = eigVectors[:,idx]
        eigValues = eigValues[idx]
        rank = np.linalg.matrix_rank(Sb)
        eigVectors = eigVectors[:,:rank]
        eigValues  = eigValues[:rank]
        self.eigenValues = eigValues
        self.W = eigVectors

    def getScatters(self, x, y):
        Si = []
        uniqueY = np.unique(y)
        for label in uniqueY:
            indexes = np.where(y == label)[0]
            selectedX = x[indexes]
            selectedXTrans = selectedX.T
            scatterMatrix = np.cov(selectedXTrans, ddof=0) * selectedXTrans.shape[1]
            Si.append(scatterMatrix)
        Sw = np.zeros(Si[0].shape)
        for scatterMatrix in Si:
            Sw += scatterMatrix
            
        xTrans = x.T
        St = np.cov(xTrans, ddof=0) * xTrans.shape[1]
        Sb = St - Sw
        
        return Sw,Sb

In [16]:
fda = FDA()
fda.fit(xTrain,yTrain)

W = fda.W

xTrainProjected = np.matmul(W.T,xTrain.T).T
xTestProjected = np.matmul(W.T,xTest.T).T

clf = LinearDiscriminantAnalysis()
clf.fit(xTrainProjected,yTrain)

print("Accuracy by own fda = ",clf.score(xTestProjected,yTest)*100,"%")




Accuracy by own fda =  72.25 %


In [19]:
yPredict = clf.predict(xTestProjected)
matrix = confusion_matrix(yTest, yPredict)
accuracyClass = matrix.diagonal()/matrix.sum(axis=1)
for i in range(10):
    print("Accuracy for class "+str(i)+" = ",accuracyClass[i]*100,"%")

Accuracy for class 0 =  74.0 %
Accuracy for class 1 =  87.0 %
Accuracy for class 2 =  53.900000000000006 %
Accuracy for class 3 =  76.3 %
Accuracy for class 4 =  61.6 %
Accuracy for class 5 =  78.9 %
Accuracy for class 6 =  36.1 %
Accuracy for class 7 =  78.2 %
Accuracy for class 8 =  85.5 %
Accuracy for class 9 =  91.0 %
